## Computational Social Science ## 

## Project 2 - Team 3 ##

### Anthony Palafox, Mahader Tamene, & Jessie Harney ### 

#### 1) Introduction ####

#### 2) Exploratory Data Analysis #####

#### 3) Preparing to Fit Models ####

#### 4) Training Models ####

#### 5) Validating & Refinding Models ####

#### 6) Discussion Questions ####